## Trabalhando com funções de agrupamento
Neste notebook iremos aprender a agrupar dados

### Importando as bibliotecas
Nesta etapa iremos apenas importar todas as bibliotecas e funções necessárias para rodar o programa

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, avg, min, max, count, approx_count_distinct, first, last, expr, collect_set, collect_list

### Criando uma SparkSession
Por meio de uma SparkSession terei acesso ao SparkContext da minha aplicação.

In [5]:
spark = SparkSession.builder.appName('Select').getOrCreate()

### Carregando os dados
Nesta etapa estamos carregando os dados que utilizaremos neste notebook

In [7]:
dados = spark \
    .read.option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .format("csv") \
    .load("/FileStore/tables/bank_additional_full-3fd09.csv")

### Usando funcoes de agrupamento

**Calcula a média de idade por job<br>
Podemos usar tanto o método .agg quanto a própria funcao de grupo**

In [10]:
dados.groupBy("job").agg(avg("age")).show()

+-------------+------------------+
 job| avg(age)|
+-------------+------------------+
 management| 42.36285909712722|
 retired|62.027325581395345|
 unknown| 45.56363636363636|
self-employed| 39.94933145672062|
 student| 25.89485714285714|
 blue-collar|39.555759671493405|
 entrepreneur|41.723214285714285|
 admin.| 38.18729610439455|
 technician| 38.5076375500519|
 services| 37.92642983119173|
 housemaid| 45.5|
 unemployed| 39.73372781065089|
+-------------+------------------+

In [11]:
dados.groupBy("job").avg("age").show()

+-------------+------------------+
 job| avg(age)|
+-------------+------------------+
 management| 42.36285909712722|
 retired|62.027325581395345|
 unknown| 45.56363636363636|
self-employed| 39.94933145672062|
 student| 25.89485714285714|
 blue-collar|39.555759671493405|
 entrepreneur|41.723214285714285|
 admin.| 38.18729610439455|
 technician| 38.5076375500519|
 services| 37.92642983119173|
 housemaid| 45.5|
 unemployed| 39.73372781065089|
+-------------+------------------+

**Acrescenta nome para a coluna agrupada (alias ou withColumnRenamed)**

In [13]:
dados.groupBy("job").agg(avg("age").alias("idade")).show()

+-------------+------------------+
 job| idade|
+-------------+------------------+
 management| 42.36285909712722|
 retired|62.027325581395345|
 unknown| 45.56363636363636|
self-employed| 39.94933145672062|
 student| 25.89485714285714|
 blue-collar|39.555759671493405|
 entrepreneur|41.723214285714285|
 admin.| 38.18729610439455|
 technician| 38.5076375500519|
 services| 37.92642983119173|
 housemaid| 45.5|
 unemployed| 39.73372781065089|
+-------------+------------------+

In [14]:
dados.groupBy("job").agg(avg("age")).withColumnRenamed("avg(age)", "idade").show()

+-------------+------------------+
 job| idade|
+-------------+------------------+
 management| 42.36285909712722|
 retired|62.027325581395345|
 unknown| 45.56363636363636|
self-employed| 39.94933145672062|
 student| 25.89485714285714|
 blue-collar|39.555759671493405|
 entrepreneur|41.723214285714285|
 admin.| 38.18729610439455|
 technician| 38.5076375500519|
 services| 37.92642983119173|
 housemaid| 45.5|
 unemployed| 39.73372781065089|
+-------------+------------------+

**Limita as 5 maiores idades**

In [16]:
dados.groupBy("job").agg(avg("age").alias("idade")).orderBy(desc("idade")).limit(5).show()

+------------+------------------+
 job| idade|
+------------+------------------+
 retired|62.027325581395345|
 unknown| 45.56363636363636|
 housemaid| 45.5|
 management| 42.36285909712722|
entrepreneur|41.723214285714285|
+------------+------------------+

**Calcula mais métricas por job**

In [18]:
dados.groupBy("job").agg(min("age").alias("min_age"), max("age").alias("max_age")).show()

+-------------+-------+-------+
 job|min_age|max_age|
+-------------+-------+-------+
 management| 21| 80|
 retired| 23| 98|
 unknown| 24| 81|
self-employed| 21| 71|
 student| 17| 47|
 blue-collar| 20| 80|
 entrepreneur| 20| 69|
 admin.| 20| 72|
 technician| 20| 70|
 services| 20| 69|
 housemaid| 21| 85|
 unemployed| 21| 66|
+-------------+-------+-------+

**Também é possível agrupar os dados por meio de dicionários**

In [20]:
aggregations = dict()
aggregations['age'] = 'min'
aggregations['pdays'] = 'max'
dados.groupBy("job").agg(aggregations).show()

+-------------+--------+----------+
 job|min(age)|max(pdays)|
+-------------+--------+----------+
 management| 21| 999|
 retired| 23| 999|
 unknown| 24| 999|
self-employed| 21| 999|
 student| 17| 999|
 blue-collar| 20| 999|
 entrepreneur| 20| 999|
 admin.| 20| 999|
 technician| 20| 999|
 services| 20| 999|
 housemaid| 21| 999|
 unemployed| 21| 999|
+-------------+--------+----------+

**Calcula countDistinct aproximado**

In [22]:
dados.groupBy("job").agg(approx_count_distinct("age", 0.1)).show()

+-------------+--------------------------+
 job|approx_count_distinct(age)|
+-------------+--------------------------+
 management| 52|
 retired| 70|
 unknown| 49|
self-employed| 45|
 student| 32|
 blue-collar| 49|
 entrepreneur| 41|
 admin.| 52|
 technician| 52|
 services| 44|
 housemaid| 62|
 unemployed| 45|
+-------------+--------------------------+

**Seleciona o primeiro o ultimo registro do dataframe**

In [24]:
dados.select(first("age"), last("age")).show()

+-----------------+----------------+
first(age, false)|last(age, false)|
+-----------------+----------------+
 56| 74|
+-----------------+----------------+

**Aplicando funcao de grupo com expr**

In [26]:
dados.select(expr("max(age)")).show()

+--------+
max(age)|
+--------+
 98|
+--------+

**Agrupamento de dados complexos. Podemos concatenar lista de valores**

In [28]:
dados.groupBy("job").agg(collect_set("education"), collect_list("education")).show()

+-------------+----------------------+-----------------------+
 job|collect_set(education)|collect_list(education)|
+-------------+----------------------+-----------------------+
 management| [university.degre...| [basic.9y, basic....|
 retired| [university.degre...| [basic.9y, high.s...|
 unknown| [university.degre...| [university.degre...|
self-employed| [university.degre...| [basic.9y, univer...|
 student| [university.degre...| [university.degre...|
 blue-collar| [university.degre...| [unknown, unknown...|
 entrepreneur| [university.degre...| [high.school, uni...|
 admin.| [university.degre...| [basic.6y, profes...|
 technician| [university.degre...| [professional.cou...|
 services| [university.degre...| [high.school, hig...|
 housemaid| [university.degre...| [basic.4y, basic....|
 unemployed| [university.degre...| [high.school, pro...|
+-------------+----------------------+-----------------------+

**Verifica a distribuicao da variavel resposta**

In [30]:
dados1 = dados.groupBy("y").agg(count("y").alias("count_class"))
total = dados1.selectExpr("sum(count_class)").collect()
dados1.select(["y", "count_class"]).withColumn("percentage", col("count_class") / lit(total[0][0])).show()

+---+-----------+-------------------+
 y|count_class| percentage|
+---+-----------+-------------------+
 no| 36548| 0.8873458288821987|
yes| 4640|0.11265417111780131|
+---+-----------+-------------------+

### Obrigado!
Quer construir uma carreira em Data Science? Acesse meu blog pessoal em https://www.hackinganalytics.com/